In [1]:
import cv2
import numpy as np
import os

class OpticalFlowCalculator:
    def __init__(self, video_path):
        self.video_path = video_path
        self.cap = cv2.VideoCapture(self.video_path)
        ret, self.first_frame = self.cap.read()
        self.prev_gray = cv2.cvtColor(self.first_frame, cv2.COLOR_BGR2GRAY)
        self.frame_count = 1  # Initialize frame count

        # Create output folder
        self.output_folder = 'optical_flow_images'
        if not os.path.exists(self.output_folder):
            os.makedirs(self.output_folder)

    def calculate_optical_flow(self, prev_gray, gray):
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])

        hsv = np.zeros_like(self.first_frame)
        hsv[..., 1] = 255
        hsv[..., 0] = (angle * 180 / np.pi / 2).astype(np.uint8)
        hsv[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

        flow_image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

        return flow_image

    def process_video(self):
        while True:
            ret, frame = self.cap.read()
            if not ret:
                break

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            flow_image = self.calculate_optical_flow(self.prev_gray, gray)
            cv2.imshow('Optical Flow', flow_image)

            image_filename = f'optical_flow_{self.frame_count:04d}.png'
            cv2.imwrite(os.path.join(self.output_folder, image_filename), flow_image)

            k = cv2.waitKey(30) & 0xff
            if k == 27 or k == ord('q'):  # 'q' key to quit
                break

            self.prev_gray = gray
            self.frame_count += 1

        self.cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    video_path = "G:\THESIS\Thoracic Rotation in Quadruped - Ask Doctor Jo - Trim.mp4"

    optical_flow_calculator = OpticalFlowCalculator(video_path)
    optical_flow_calculator.process_video()


error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
